In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widjets
%matplotlib inline

import numpy as np
import collections

from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt

import os
import shutil
import cv2
from pathlib import Path
from copy import deepcopy

from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from time import time
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn import metrics
import json

from scipy.stats import multivariate_normal

In [2]:
from typing import Callable, List, Tuple

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import catalyst
from catalyst import utils

print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU

SEED = 42
utils.set_global_seed(SEED)
utils.prepare_cudnn(deterministic=True)

torch: 1.7.0, catalyst: 20.08.2


In [11]:
def Load_train_data():
    #X_test = np.load('idchess_task/xtest.npy')
    X_train = np.load(r'Z:\WORK\test tasks\idchess\idchess_task/xtrain.npy').astype(np.uint8)
    Y_train = np.load(r'Z:\WORK\test tasks\idchess\idchess_task/ytrain.npy')

    #preprocess Y_train
    Y_train = np.around(np.clip(Y_train*255, 0, 255)).astype(int).reshape(-1,4,2)

    print(Y_train.shape, X_train.shape)
    return X_train, Y_train


def Get_Gauss_point():
    """
    Get Gauss blurred point on 25x25 grid
    """
    x, y = np.mgrid[0:1.0:25j, 0:1.0:25j]
    xy = np.column_stack([x.flat, y.flat])
    mask = np.zeros(25 * 25)
    COV=np.diag([0.03,0.03])
    Mean=[0.5,0.5]
    Gauss = multivariate_normal.pdf(xy, mean=Mean, cov=COV)
    Gauss *= np.sqrt(np.linalg.det(COV)*(2*np.pi)**2)  #normalize to 1(formulae Gauss)
    Gauss = Gauss.reshape((25, 25))
    return Gauss

Gauss = Get_Gauss_point()


def Get_mask(points):
    """
    Get mask for image

    Parameters
    ----------
    points : ndarray of shape [n_samples, 1, 2]
        array with point coordinates shaped [x,y]
    Returns
        -------
        mask:
            256x256 mask with gauss blurred dots
        """
    points = points.astype(int)
    global Gauss
    mask = np.zeros((256+12*2,256+12*2)) #центр гауссианы смещен влево вверх по 12 пикселей, 
                                    #поэтому можно расширить маску по 12 с каждой стороны и потом обрезать
    for point in points:
        x,y = point
        mask[y:y+25,x:x+25] += Gauss
        
    mask = mask[12:-12,12:-12]
    np.clip(mask, 0, 1)
    return mask

def Get_coords(mask):
    """
    get coordinates of angles of chess board
    
    Parameters
    ----------
    mask : ndarray of shape [height, width]
        predicted mask 
    Returns
        -------
        coords:
            ndarray of shape [n_dots, 1, 2]
    """
    ret, labels = cv2.connectedComponents((mask>0.9).astype(np.uint8))
    predicted_points = []
    for i in range(1,ret+1):
        dots = np.argwhere(labels==i)
        N = dots.shape[0]
        if N>=4: #если  "кучка точек состоит из более чем 4 пикселей"
            coord = (dots.sum(axis=0)/N).astype(int)
            predicted_points.append(coord)
    
    #assert len(predicted_points)==4, f'num of predicted dots is {len(predicted_points)}'
    return np.vstack(predicted_points)

In [12]:
class SegmentationDataset(Dataset):
    def __init__(
        self,
        images,
        coords = None,
        transforms=None
    ) -> None:
        self.images = images
        self.coords = coords
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx: int) -> dict:
        img = self.images[idx]
        image = np.concatenate((img,img,img),axis=2)
        result = {"image": image}
        
        if self.coords is not None:
            mask = Get_mask(self.coords[idx])
            result["mask"] = mask
        
        if self.transforms is not None:
            result = self.transforms(**result)

        return result

In [13]:
from albumentations import (
    HorizontalFlip, RandomRotate90, Normalize, Compose, HueSaturationValue,
    ShiftScaleRotate, RandomGamma, IAAAdditiveGaussianNoise, GaussNoise,
    JpegCompression, Flip, Transpose, MedianBlur,
    IAASharpen, IAAEmboss, RandomBrightnessContrast,
    MotionBlur, OneOf, Blur, CLAHE, CropNonEmptyMaskIfExists,
    ElasticTransform, GridDistortion, OpticalDistortion, IAAAffine)
from albumentations.pytorch.transforms import ToTensor

def post_transform():
    return Compose([
        Normalize(),
        ToTensor()
    ])

def train_transforms(p = 0.3):
    transforms = [
        RandomRotate90(p=p),
        Flip(p=p),
        Transpose(p=p),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        CLAHE(p=p),
        IAASharpen(p=p),
        IAAEmboss(p=p),
        RandomBrightnessContrast(
            brightness_limit=0.5,
            contrast_limit=0.5,
            p=p
        ),
        RandomGamma(gamma_limit=(85, 115), p=p),
        JpegCompression(quality_lower=75, p=p),
        post_transform()
    ]
    transforms = Compose(transforms)
    return transforms


def valid_transforms():
    return post_transform()

In [14]:
# На посмотреть
X_train, Y_train = Load_train_data()
train_tr = train_transforms()

@interact(i=(0,len(X_train)-1,1))
def tst(i=0):
    points = Y_train[i].copy()
    img = X_train[i]
    image = np.concatenate((img,img,img),axis=2)
    result = {'image': image, 'mask':Get_mask(Y_train[i].copy())}
    output = train_tr(**result)
    plt.subplot(1,2,1)
    plt.imshow(output['image'][0])
    plt.subplot(1,2,2)
    plt.imshow(output['mask'][0])
    

(15137, 4, 2) (15137, 256, 256, 1)


interactive(children=(IntSlider(value=0, description='i', max=15136), Output()), _dom_classes=('widget-interac…

In [6]:
def get_loaders(
    random_state: int,
    valid_size: float = 0.2,
    batch_size: int = 32,
    num_workers: int = 4,
    train_transforms_fn = None,
    valid_transforms_fn = None,
) -> dict:

    X, Y = Load_train_data()
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)
    

    # Creates our train dataset
    train_dataset = SegmentationDataset(
      images = X_train,#.tolist(),
      coords = Y_train,
      transforms = train_transforms_fn
    )

    # Creates our valid dataset
    valid_dataset = SegmentationDataset(
      images = X_val,
      coords = Y_val,
      transforms = valid_transforms_fn
    )

    # Catalyst uses normal torch.data.DataLoader
    train_loader = DataLoader(
      train_dataset,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      drop_last=True,
    )

    valid_loader = DataLoader(
      valid_dataset,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      drop_last=True,
    )

    # And excpect to get an OrderedDict of loaders
    loaders = collections.OrderedDict()
    loaders["train"] = train_loader
    loaders["valid"] = valid_loader

    return loaders

# TRAIN MODEL

In [23]:
batch_size=24
loaders = get_loaders(
    random_state=SEED,
    train_transforms_fn=train_transforms(),
    valid_transforms_fn=post_transform(),
    batch_size=batch_size,
    num_workers=0
)

(15137, 4, 2) (15137, 256, 256, 1)


In [9]:
import segmentation_models_pytorch as smp

# We will use Feature Pyramid Network with pre-trained ResNeXt50 backbone
model = smp.FPN(encoder_name="efficientnet-b1", classes=1)


In [10]:
from catalyst.contrib.nn import DiceLoss, IoULoss

# we have multiple criterions
criterion = {
    "dice": DiceLoss(),
    "iou": IoULoss(),
    "bce": nn.BCEWithLogitsLoss()
}

from torch import optim

from catalyst.contrib.nn import RAdam, Lookahead

learning_rate = 0.001
encoder_learning_rate = 0.0005

# Since we use a pre-trained encoder, we will reduce the learning rate on it.
layerwise_params = {"encoder*": dict(lr=encoder_learning_rate, weight_decay=0.00003)}

# This function removes weight_decay for biases and applies our layerwise_params
model_params = utils.process_model_params(model, layerwise_params=layerwise_params)

# Catalyst has new SOTA optimizers out of box
base_optimizer = RAdam(model_params, lr=learning_rate, weight_decay=0.0003)
optimizer = Lookahead(base_optimizer)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.25, patience=2)


In [11]:
from catalyst.dl import SupervisedRunner

num_epochs = 10
logdir = "./logs/segmentation"

device = utils.get_device()
print(f"device: {device}")


# by default SupervisedRunner uses "features" and "targets",
# in our case we get "image" and "mask" keys in dataset __getitem__
runner = SupervisedRunner(device=device, input_key="image", input_target_key="mask")

device: cuda


In [12]:
%load_ext tensorboard
%tensorboard --logdir {logdir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6724), started 0:00:39 ago. (Use '!kill 6724' to kill it.)

In [12]:
from catalyst.dl import DiceCallback, IouCallback, \
  CriterionCallback, MetricAggregationCallback

callbacks = [
    # Each criterion is calculated separately.
    CriterionCallback(
        input_key="mask",
        prefix="loss_dice",
        criterion_key="dice"
    ),
    CriterionCallback(
        input_key="mask",
        prefix="loss_iou",
        criterion_key="iou"
    ),
    CriterionCallback(
        input_key="mask",
        prefix="loss_bce",
        criterion_key="bce"
    ),

    # And only then we aggregate everything into one loss.
    MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum", # can be "sum", "weighted_sum" or "mean"
        # because we want weighted sum, we need to add scale for each loss
        metrics={"loss_dice": 1.0, "loss_iou": 1.0, "loss_bce": 0.8},
    ),

    # metrics
    DiceCallback(input_key="mask"),
    IouCallback(input_key="mask"),
]

model.train()
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    # our dataloaders
    loaders=loaders,
    # We can specify the callbacks list for the experiment;
    callbacks=callbacks,
    # path to save logs
    logdir=logdir,
    num_epochs=num_epochs,
    # save our best checkpoint by IoU metric
    main_metric="dice",
    # IoU needs to be maximized.
    minimize_metric=False,
    # prints train logs
    verbose=True,
)

1/10 * Epoch (train):   0% 1/536 [00:02<25:47,  2.89s/it, dice=0.014, iou=0.007, loss=3.083, loss_bce=1.379, loss_dice=0.986, loss_iou=0.993]

..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning:

This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha)



1/10 * Epoch (train): 100% 536/536 [05:46<00:00,  1.55it/s, dice=0.542, iou=0.372, loss=1.110, loss_bce=0.029, loss_dice=0.458, loss_iou=0.628]
1/10 * Epoch (valid): 100% 94/94 [00:18<00:00,  5.19it/s, dice=0.582, iou=0.410, loss=1.029, loss_bce=0.026, loss_dice=0.418, loss_iou=0.590]
[2020-11-21 20:51:20,727] 
1/10 * Epoch 1 (_base): lr=0.0005 | momentum=0.9000
1/10 * Epoch 1 (train): dice=0.3991 | iou=0.2633 | loss=1.3787 | loss_bce=0.0514 | loss_dice=0.6009 | loss_iou=0.7367
1/10 * Epoch 1 (valid): dice=0.5910 | iou=0.4194 | loss=1.0100 | loss_bce=0.0254 | loss_dice=0.4090 | loss_iou=0.5806
2/10 * Epoch (train): 100% 536/536 [05:40<00:00,  1.57it/s, dice=0.602, iou=0.430, loss=0.991, loss_bce=0.029, loss_dice=0.398, loss_iou=0.570]
2/10 * Epoch (valid): 100% 94/94 [00:18<00:00,  5.18it/s, dice=0.614, iou=0.443, loss=0.963, loss_bce=0.026, loss_dice=0.386, loss_iou=0.557]
[2020-11-21 20:57:19,771] 
2/10 * Epoch 2 (_base): lr=0.0005 | momentum=0.9000
2/10 * Epoch 2 (train): dice=0.576

# TEST STAGE

In [7]:
import collections

from catalyst.dl import Callback, CallbackOrder, IRunner

class CustomInferCallback(Callback):
    def __init__(self, path=Path(r'logs\test_output')):
        super().__init__(CallbackOrder.Internal)
        self.path = path
        
    def on_loader_start(self, runner: IRunner):
        self.file_counter = 0
        self.errors = []

    def on_batch_end(self, runner: IRunner):
        # data from the Dataloader
        # image, mask = runner.input["image"], runner.input["mask"]
        
        #Get_coords(mask)
        inputs = runner.input["image"].detach().cpu()
        images = utils.tensor_to_ndimage(inputs)
        
        output_masks = runner.output["logits"].sigmoid().detach().cpu().numpy()
        
        print(runner.output.keys())
        for img, mask in zip(images,output_masks):
            points = Get_coords(mask[0])
            img = cv2.UMat(img)
            for point in points:
                cv2.circle(img, (point[1],point[0]), 10, (0,1,0), -1)
                
            f_name = str(self.file_counter).zfill(4)
            img = np.around(img.get()*255).astype(np.uint8)
            cv2.imwrite(str(self.path/f_name)+'.png',img)
            
            self.file_counter += 1
            if len(points) != 4:
                self.errors.append(f_name)
                
                
    def on_loader_end(self, runner: IRunner):
        print('Prediction errors in this files:')
        print(self.errors)

In [8]:
test_dataset = SegmentationDataset(
      images = np.load('idchess_task/xtest.npy'),
      transforms = post_transform()
    )

test_loader = DataLoader(
      test_dataset,
      batch_size=1,
      shuffle=None,
      num_workers=0,
    )

import segmentation_models_pytorch as smp
from collections import OrderedDict
from catalyst.dl import CheckpointCallback
from catalyst.dl import SupervisedRunner

logdir = "./logs/segmentation"
infer_loaders = {"test": test_loader}
#infer_loaders = {"test": loaders['valid']}
model = smp.FPN(encoder_name="efficientnet-b1", classes=1)

runner = SupervisedRunner(device=utils.get_device(), input_key="image", input_target_key="mask")
runner.infer(
    model=model,
    loaders=infer_loaders,
    callbacks=OrderedDict([
        ("loader", CheckpointCallback(resume=f"{logdir}/checkpoints/best.pth")),
        ("test", CustomInferCallback())
    ]),
)

=> Loading checkpoint ./logs/segmentation/checkpoints/best.pth
loaded state checkpoint ./logs/segmentation/checkpoints/best.pth (global epoch 10, epoch 10, stage train)
dict_keys(['logits'])
dict_keys(['logits'])
dict_keys(['logits'])
dict_keys(['logits'])
dict_keys(['logits'])
Prediction errors in this files:
[]
